# Analiza wykorzystania różnych heurystyk przy rozwiązaniu problemu komiwojażera

Grupa: Amelia Madej, Justyna Sarkowicz, Olga Sieradzan, Weronika Duda i Aleksandra Węgrzyn

## Biblioteki

In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
 

## Pobór danych 

In [3]:
#file_path1 = "C:\Users\Justyna\source\repos\Projekt_IO\TSP-problem/Dane_TSP_48.xlsx"
#file_path2 = "C:/Users/olgas/OneDrive/Documents/GitHub/IO/Projekt I/Dane_TSP_76.xlsx"
#file_path3 = "C:/Users/olgas/OneDrive/Documents/GitHub/IO/Projekt I/Dane_TSP_127.xlsx"

file_path1 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_48.xlsx"
file_path2 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_76.xlsx"
file_path3 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()

data2 = pd.read_excel(file_path2, index_col=0).to_numpy()

data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Algorytm najbliższego sąsiada NN

In [4]:

# Klasyczny algorytm NN

def nearest_neighbor(matrix, start_point=0, max_time=60, random_restart=1):
    n = len(matrix)
    best_path, best_cost = None, float('inf')
    start_time = time.time()

    for _ in range(random_restart):
        # Wybór losowego punktu startowego przy wielu restartach
        current_start_point = np.random.randint(0, n) if random_restart > 1 else start_point

        # Przygotowanie zmiennych dla algorytmu NN
        visited = [False] * n
        path = [current_start_point]
        visited[current_start_point] = True

        # Generowanie ścieżki metodą najbliższego sąsiada
        while len(path) < n:
            last = path[-1]
            next_city = np.argmin([
                matrix[last][j] if not visited[j] else np.inf
                for j in range(n)
            ])
            path.append(next_city)
            visited[next_city] = True

        # Oblicz koszt uzyskanej ścieżki
        cost = calculate_path_cost(matrix, path)

        # Sprawdź, czy znaleziono lepsze rozwiązanie
        if cost < best_cost:
            best_path, best_cost = path, cost

        # Przerwij, jeśli przekroczono limit czasu
        if time.time() - start_time > max_time:
            break

    return best_path, best_cost



# Wybieramy dwie pozycje i zmieniamy je miejscami 
def swap_move(path):  
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:]
        new_path[i], new_path[j] = new_path[j], new_path[i]
        yield new_path
 

# Odwracamy segment pomiędzy wybranymi wartościami czyli 
# [a b c d e] -> [a d c b e]
def two_opt_move(path):
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:i] + path[i:j][::-1] + path[j:]
        yield new_path
 
# Wybieramy jeden wierzchołek w ścierzce, usuwamy z jednego miejsca i przerzucamy w inne miejsce
def insertion_move(path):
    n = len(path)
    for i in range(n):
        for j in range(n):
            if i != j:
                new_path = path[:]
                node = new_path.pop(i)
                new_path.insert(j, node)
                yield new_path

# Policzenie łącznej odległości między startem a końcem                
def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]
 
# Optymalizacja rozwiązania 
# Argumenty: matrix odległości / dotychczasowe rozwiązanie / ['swap', '2-opt', 'insertion'] / maksymalna liczba iteracji
def local_search(matrix, initial_path, move_generators, max_iterations):
    best_path = initial_path
    best_cost = calculate_path_cost(matrix, best_path)
 
    iteration = 0
    improved = True
    while improved and iteration < max_iterations:
        improved = False
        for move_generator in move_generators:
            for new_path in move_generator(best_path):
                new_cost = calculate_path_cost(matrix, new_path)
                if new_cost < best_cost:
                    best_path, best_cost = new_path, new_cost
                    improved = True
                    break  
            if improved:
                break
        iteration += 1
 
    return best_path, best_cost

# Ostateczna funkcja 

#  Parametry:
# file_path (str): Ścieżka do pliku Excel zawierającego macierz odległości.
# start_point (int): Miasto początkowe dla algorytmu.
# max_time (int): Maksymalny czas dozwolony na obliczenia (w sekundach).
# move_types (list): Lista typów ruchów dla wyszukiwania lokalnego (np. ['swap', '2-opt', 'insertion']).
# random_restart (int): Liczba losowych restartów w celu wyjścia poza lokalne minima, 
#                       przy każdej iteracji wybieramy nowy start_point a nie cały czas ten sam co ustawiony, jeżeli  random_restart > 1
# max_iterations (int): Maksymalna liczba iteracji dla wyszukiwania lokalnego.
 
# Zwraca:
# best_path (list): Znaleziono optymalną ścieżkę.
# best_cost (float): Długość optymalnej ścieżki.

# Jeżeli move_types=None to używamy wszytskich sposobów do optymalizacji

def solve_tsp(matrix, start_point=0, max_time=60, move_types=None, random_restart=1, max_iterations=100):
   
    
    move_generators = []
    if move_types is None:
        move_types = ['swap', '2-opt', 'insertion']  
    if 'swap' in move_types:
        move_generators.append(swap_move)
    if '2-opt' in move_types:
        move_generators.append(two_opt_move)
    if 'insertion' in move_types:
        move_generators.append(insertion_move)
 
    best_path, best_cost = None, float('inf')
    start_time = time.time()

    # Iteracje z różnym startem
    for _ in range(random_restart):
        # wybranie punktu startującego 
        initial_start_point = np.random.randint(0, len(matrix)) if random_restart > 1 else start_point
 
        # Początkowo generujemy rożwiązanie za pomocą klasycznego NN
        initial_path, initial_cost = nearest_neighbor(matrix, start_point=initial_start_point)
 
        #print(f"Initial NN path (start {initial_start_point}): {initial_path} with cost: {initial_cost}")
 
        # Potem przeprowadzamy optymalizacje z pomocą innych parametrów
        current_path, current_cost = local_search(matrix, initial_path, move_generators, max_iterations)
 
        # Aktualizacja najlepszego wyniku
        if current_cost < best_cost:
            best_path, best_cost = current_path, current_cost
        
 
        # Zabezpieczenie przed przekroczeniem czasu 
        if time.time() - start_time > max_time:
            break
 
    return best_path, best_cost
 

# przykład użycia 
best_path, best_cost = solve_tsp(
    data1, 
    start_point=0, 
    max_time=60, 
    move_types='swap', 
    random_restart=1, 
    max_iterations=150
)


## Generowanie rozwiązań 

Za podstawowe dane przyjmujemy :

* start_ point = 0 

* max_time = 500

* random_restart = 1 

* max_itaration = 150 

Nastepnie badane są wpływy poszczególnych parametrów na każdą metodę 

In [28]:
# Klasyczna metoda NN 
classic_p_1, classic_c_1 = nearest_neighbor(data1, start_point=0 , max_time=500, random_restart=1)
classic_p_2, classic_c_2 = nearest_neighbor(data2, start_point=0, max_time=500, random_restart=1)
classic_p_3, classic_c_3 = nearest_neighbor(data3, start_point=0, max_time=500, random_restart=1)

In [29]:
# Move type = 'swap'

swap_p_1, swap_c_1 = solve_tsp(
    data1, 
    start_point=0, 
    max_time=500, 
    move_types='swap', 
    random_restart=1, 
    max_iterations=150
)

swap_p_2, swap_c_2 = solve_tsp(
    data2, 
    start_point=0, 
    max_time=500, 
    move_types='swap', 
    random_restart=1, 
    max_iterations=150
)

swap_p_3, swap_c_3 = solve_tsp(
    data3, 
    start_point=0, 
    max_time=500, 
    move_types='swap', 
    random_restart=1, 
    max_iterations=150
)

In [30]:
# Move typ = "2 opt"

opt_p_1, opt_c_1 = solve_tsp(
    data1, 
    start_point=0, 
    max_time=500, 
    move_types= '2-opt', 
    random_restart=1, 
    max_iterations=150
)

opt_p_2, opt_c_2 = solve_tsp(
    data2, 
    start_point=0, 
    max_time=500, 
    move_types= '2-opt', 
    random_restart=1, 
    max_iterations=150
)

opt_p_3, opt_c_3 = solve_tsp(
    data3, 
    start_point=0, 
    max_time=500, 
    move_types= '2-opt', 
    random_restart=1, 
    max_iterations=150
)

In [31]:
# Move typ = "insertion"

ins_p_1, ins_c_1 = solve_tsp(
    data1, 
    start_point=0, 
    max_time=500, 
    move_types= 'insertion', 
    random_restart=1, 
    max_iterations=150
)

ins_p_2, ins_c_2 = solve_tsp(
    data2, 
    start_point=0, 
    max_time=500, 
    move_types= 'insertion', 
    random_restart=1, 
    max_iterations=150
)

ins_p_3, ins_c_3 = solve_tsp(
    data3, 
    start_point=0, 
    max_time=500, 
    move_types= 'insertion', 
    random_restart=1, 
    max_iterations=150
)

Zebranie wyników z podstawowymi parametrami 

In [32]:
W1 =  {
    "Długość ścieżki": [classic_c_1, swap_c_1, opt_c_1, ins_c_1]
}

c1 = pd.DataFrame(data = W1)
c1.index = ["Klasyczna","Swap" , "2-opt", "Insertion"]

W2 =  {
    "Długość ścieżki": [classic_c_2, swap_c_2, opt_c_2, ins_c_2]
}

c2 = pd.DataFrame(data = W2)
c2.index = ["Klasyczna","Swap" , "2-opt", "Insertion"]

W3 =  {
    "Długość ścieżki": [classic_c_3, swap_c_3, opt_c_3, ins_c_3]
}

c3 = pd.DataFrame(data = W3)
c3.index = ["Klasyczna","Swap" , "2-opt", "Insertion"]

Badanie wpływu parametru MAX ITTERATIONS

In [33]:
# Stałe parametry
start_point = 0
max_time = 500
random_restart = 1

# DANE NR 1 
results = []
for max_iterations in range(1, 301):
    _, best_cost = solve_tsp(data1, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})


itterration1 = pd.DataFrame(results)


# DANE NR 2
results = []
for max_iterations in range(1, 301):
    _, best_cost = solve_tsp(data2, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})

# Tworzenie ramki danych
itterration2 = pd.DataFrame(results)


# DANE NR 3
results = []
for max_iterations in range(1, 301):
    _, best_cost = solve_tsp(data3, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})

# Tworzenie ramki danych
itterration3 = pd.DataFrame(results)

Badanie wpływu parametru random restart

In [34]:
# Stałe parametry
start_point = 0
max_time = 500
max_iterations = 150

# DANE NR 1 
results = []
for random_restart in range(1, 31):
    _, best_cost = solve_tsp(data1, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": random_restart, "WYNIK": best_cost})


restar1 = pd.DataFrame(results)


# DANE NR 2
results = []
for random_restart in range(1, 31):
    _, best_cost = solve_tsp(data2, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": random_restart, "WYNIK": best_cost})

# Tworzenie ramki danych
restart2 = pd.DataFrame(results)


# DANE NR 3
results = []
for random_restart in range(1, 31):
    _, best_cost = solve_tsp(data3, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": random_restart, "WYNIK": best_cost})

# Tworzenie ramki danych
restart3 = pd.DataFrame(results)

Wystartowanie z 4 różnych losowych puntktów startowych

In [50]:
# Stałe parametry
start_points = [np.random.randint(0, len(data1)), np.random.randint(0, len(data1)), np.random.randint(0, len(data1)), np.random.randint(0, len(data1))]
max_time = 500
max_iterations = 150
random_restart = 1

# DANE NR 1 
results = []
for start_point in start_points:
    _, best_cost = solve_tsp(data1, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": start_point, "WYNIK": best_cost})


point1 = pd.DataFrame(results)

start_points = [np.random.randint(0, len(data2)), np.random.randint(0, len(data2)), np.random.randint(0, len(data2)), np.random.randint(0, len(data2))]

# DANE NR 2
results = []
for start_point in start_points:
    _, best_cost = solve_tsp(data2, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": start_point, "WYNIK": best_cost})

# Tworzenie ramki danych
point2 = pd.DataFrame(results)

start_points = [np.random.randint(0, len(data3)), np.random.randint(0, len(data3)), np.random.randint(0, len(data3)), np.random.randint(0, len(data3))]
# DANE NR 3
results = []
for start_point in start_points:
    _, best_cost = solve_tsp(data3, start_point=start_point, max_time=max_time, random_restart=random_restart, max_iterations=max_iterations)
    results.append({"PARAMETR": start_point, "WYNIK": best_cost})

# Tworzenie ramki danych
point3 = pd.DataFrame(results)

Zapis wyników w pliku EXCEL

In [51]:
res = {
    "Porównanie_metod+1": c1,
    "max_iterations_1": itterration1,
    "Restart_1": restar1,
    "Point_1" : point1,
    "Porównanie_metod_2": c2,
    "max_iterations_2": itterration2,
    "Restart_2": restart2,
    "Point_2" : point2,
    "Porównanie_metod_3": c3,
    "max_iterations_3": itterration3,
    "Restart_3": restart3,
    "Point_3" : point3,
    
}

# Ścieżka do pliku Excel
file_name = "NN.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in res.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")

Wyniki zostały zapisane w pliku NN.xlsx


## Algorytm wspinaczki z multistartem (IHC)

## Algorytm symulowanego wyżarzania (SA)

## Algorytm przeszukiwania Tabu 

## Algorytm genetyczny

## Algorytm zachłanny

In [42]:
## Klasyczny algorytm zachłanny 
def tsp_greedy(distance_matrix, start=0):
    n = len(distance_matrix)
    visited = [False] * n
    visited[start] = True
    path = [start]
    total_cost = 0
    current_city = start

    for _ in range(n - 1):
        # Znajdujemy najbliższe miasto którego nie odwiedziliśmy 
        next_city = min(
            (city for city in range(n) if not visited[city]),
            key=lambda city: distance_matrix[current_city][city]
        )
        # Dodajemy odległośc i zaznaczamy ze miasto jest już odwiedzone
        total_cost += distance_matrix[current_city][next_city]
        visited[next_city] = True
        path.append(next_city)
        current_city = next_city

    # Obliczanie końcowej trasy
    total_cost += distance_matrix[current_city][start]
    path.append(start)

    return path, total_cost

## Iteracja algorytmem zachłannym po każdym możliwym punkcie startowym 

def tsp_greedy_all_starts(distance_matrix):
    best_cost = float('inf')
    best_path = None

    for start in range(len(distance_matrix)):
        path, cost = tsp_greedy(distance_matrix, start=start)
        if cost < best_cost:
            best_cost = cost
            best_path = path
            point = start

    return best_cost, best_path, point

## Generowanie rozwiazań

In [43]:
# Klasyczny ( od punktu 1 )
classic_p_1, classic_c_1 = tsp_greedy(data1)
classic_p_2, classic_c_2 = tsp_greedy(data2)
classic_p_3, classic_c_3 = tsp_greedy(data3)

In [44]:
# Start we wszytskich punktach 
all_p_1, all_c_1, all_point_1= tsp_greedy_all_starts(data1)
all_p_2, all_c_2, all_point_2 = tsp_greedy_all_starts(data2)
all_p_3, all_c_3, all_point_3 = tsp_greedy_all_starts(data3)

Zebranie wyników oraz zapis w pliku EXCEL 

In [52]:
W1 =  {
    "Długość ścieżki": [classic_c_1,classic_c_2,classic_c_3 ]
}

p1 = pd.DataFrame(data = W1)
p1.index = ["Dane 1","Dane 2" , "Dane 3", ]

W2 =  {
    "Długość ścieżki": [all_p_1, all_p_2, all_p_3],
    "Punkt": [all_point_1,all_point_2, all_point_3]
}

p2 = pd.DataFrame(data = W2)
p2.index = ["Dane 1","Dane 2" , "Dane 3"]

resu = {
    "Klasyka": p1,
    "Wszytskie":p2    
}

# Ścieżka do pliku Excel
file_name = "Zachłanny.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in resu.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")

Wyniki zostały zapisane w pliku Zachłanny.xlsx
